## Method3 (Basically, Method2 but without bbox)

In [20]:
import trimesh
import pyvista as pv
from utils import *

#### Load a file

In [21]:
mesh = pymesh.load_mesh("data/two_spheres2.ply")

In [22]:
self_intersection_stats(mesh)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     952 |
+-----------------------------------+---------+
| Number of faces                   |    1896 |
+-----------------------------------+---------+
| Number of intersecting face pairs |     119 |
+-----------------------------------+---------+


In [23]:
visualize_intersection(mesh)

Widget(value='<iframe src="http://localhost:40071/index.html?ui=P_0x7f3eb48f2dd0_1&reconnect=auto" class="pyvi…

None

#### Save intersection info first

In [24]:
intersections = pymesh.detect_self_intersection(mesh)
intersecting_vertices, intersecting_faces = track_self_intersecting_faces(mesh, intersections)

#### Get outer hull

In [25]:
outer_hull = pymesh.compute_outer_hull(mesh)

In [26]:
self_intersection_stats(outer_hull)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     917 |
+-----------------------------------+---------+
| Number of faces                   |    1830 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


#### Map intersecting region to modified mesh & Extract submesh

In [27]:
mapped_vertices = map_to_modified_mesh(mesh, outer_hull, intersecting_vertices)

In [28]:
submesh, face_mask = extract_self_intersecting_region_from_modified(outer_hull, mapped_vertices)

In [10]:
visualize(submesh, filename="Submesh")

Widget(value='<iframe src="http://localhost:38305/index.html?ui=P_0x7f9029eedf90_1&reconnect=auto" class="pyvi…

None

In [29]:
remaining_mesh = extract_remaining_mesh(outer_hull, face_mask)

In [12]:
visualize(remaining_mesh, filename="remaining_mesh")

Widget(value='<iframe src="http://localhost:38305/index.html?ui=P_0x7f8fd76b2620_2&reconnect=auto" class="pyvi…

None

#### Repair (or Remesh) the submesh
- `pymesh.collapse_short_edges(mesh, abs_threshold, rel_threshold, preserve_feature)`<br>
Collapses all edges with length less than a user specified threshold.

- `pymesh.split_long_edges(mesh, max_edge_length)`<br>
Splits long edges into 2 or more shorter edges.

- `pymesh.remove_obtuse_triangles(mesh, max_angle, max_iterations)`<br>
Splits each obtuse triangle into 2 or more right or sharp triangles.

- `pymesh.remove_degenerated_triangles(mesh, num_iterations)` <br>
Degenerate triangles are triangles with collinear vertices (i.e., some vertices are lying in the same straight line). They have zero areas and their normals are undefined.

In [30]:
from numpy.linalg import norm

def method2(mesh, detail="low"):

    bbox_min, bbox_max = mesh.bbox
    diag_len = norm(bbox_max - bbox_min)
    if detail == "normal":
        target_len = diag_len * 5e-3
    elif detail == "high":
        target_len = diag_len * 2.5e-3
    elif detail == "low":
        target_len = diag_len * 2e-2
    print("Target resolution: {} mm".format(target_len))

    count = 0
    mesh, __ = pymesh.remove_degenerated_triangles(mesh, 100)
    mesh, __ = pymesh.split_long_edges(mesh, target_len)
    num_vertices = mesh.num_vertices
    while True:
        mesh, __ = pymesh.collapse_short_edges(mesh, 1e-6, preserve_feature=True) # Remove extremely small edges
        mesh, __ = pymesh.collapse_short_edges(mesh, target_len,
                                               preserve_feature=True)
        mesh, __ = pymesh.remove_obtuse_triangles(mesh, 150.0, 100)
        if mesh.num_vertices == num_vertices:
            break

        num_vertices = mesh.num_vertices
        print("#v: {}".format(num_vertices))
        count += 1
        if count > 10: break

    mesh = pymesh.resolve_self_intersection(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh = pymesh.compute_outer_hull(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh, __ = pymesh.remove_obtuse_triangles(mesh, 179.0, 5)
    mesh, __ = pymesh.remove_isolated_vertices(mesh)

    return mesh

In [31]:
repaired_submesh = method2(submesh)

Target resolution: 0.058262021377178466 mm
#v: 428
#v: 427


In [15]:
visualize(repaired_submesh, filename="Repaired_submesh")

Widget(value='<iframe src="http://localhost:38305/index.html?ui=P_0x7f8fb50fd240_3&reconnect=auto" class="pyvi…

None

#### Align submesh
Since we have modified the submesh, we need to change its boundary to what it used to be. Otherwise, when we combine the submesh and the remaining component later, there will be some small gaps.

In [32]:
aligned_submesh = align_submesh_boundary(remaining_mesh, repaired_submesh)

In [17]:
visualize(aligned_submesh, filename="Method2")

Widget(value='<iframe src="http://localhost:38305/index.html?ui=P_0x7f8fb50fe4d0_4&reconnect=auto" class="pyvi…

None

#### Merge two meshes

In [33]:
repaired_full = replace_submesh_in_original(remaining_mesh, aligned_submesh)

In [19]:
visualize(repaired_full)

Widget(value='<iframe src="http://localhost:38305/index.html?ui=P_0x7f8f94b2ee60_5&reconnect=auto" class="pyvi…

None

#### Final refinement

In [34]:
final, _ = pymesh.remove_duplicated_vertices(repaired_full)
final, _ = pymesh.remove_duplicated_faces(final)
final, _ = pymesh.remove_degenerated_triangles(final)
final = pymesh.compute_outer_hull(final)
final, __ = pymesh.collapse_short_edges(final, 1e-6)
final, __ = pymesh.remove_obtuse_triangles(final, 175.0, 10)
final, _ = pymesh.remove_isolated_vertices(final)
final = pymesh.resolve_self_intersection(final)

In [35]:
self_intersection_stats(final)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |    1021 |
+-----------------------------------+---------+
| Number of faces                   |    2040 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


In [ ]:
visualize(final, filename="Final Outerhull")

In [36]:
evaluation(outer_hull, final)

+-----------------------------------+------------+-----------+
| Metric                            |     Before |     After |
+===================================+============+===========+
| Number of vertices                |  917       | 1021      |
+-----------------------------------+------------+-----------+
| Number of faces                   | 1830       | 2040      |
+-----------------------------------+------------+-----------+
| Number of intersecting face pairs |    0       |    0      |
+-----------------------------------+------------+-----------+
| Volume                            |    6.61368 |    6.6111 |
+-----------------------------------+------------+-----------+
| Area                              |   19.0834  |   19.0837 |
+-----------------------------------+------------+-----------+


In [ ]:
evaluate_displacement(outer_hull, final)

In [37]:
pymesh.save_mesh("temp.ply", final, ascii=True)